In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip -qq 'Cat_Dog_data.zip'

--2019-12-27 08:22:49--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.111.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.111.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580495262 (554M) [application/zip]
Saving to: ‘Cat_Dog_data.zip’

Cat_Dog_data.zip    100%[===================>] 553.60M  16.3MB/s    in 35s     

2019-12-27 08:23:25 (15.7 MB/s) - ‘Cat_Dog_data.zip’ saved [580495262/580495262]



In [0]:
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [0]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt 
import torch 
from torchvision import datasets, transforms 

train_transform = transforms.Compose([transforms.RandomRotation(180),
                                 transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

ROOT = '/content/Cat_Dog_data/'

train_data = datasets.ImageFolder(ROOT + 'train', transform = train_transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

test_data = datasets.ImageFolder(ROOT + 'test', transform = test_transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

In [5]:
images, labels = next(iter(trainloader))
print(images.shape)

torch.Size([32, 3, 224, 224])


In [15]:
from torchvision import models 
import torch.nn as nn 

model = models.densenet121(pretrained=True)
print(model)
classifier = nn.Sequential(nn.Linear(1024, 500),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(500, 2),
                           nn.LogSoftmax(dim = 1))

model.classifier = classifier

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [16]:
import time
import torch.optim as optim 
from tqdm import tqdm 

tmp = time.time()
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")
print(f"Total time processing : {time.time()-tmp:.3f} seconds")

Device = cpu; Time per batch: 7.115 seconds
Device = cuda; Time per batch: 0.029 seconds
Total time processing : 86.770 seconds


In [17]:
import torch.optim as optim 
from tqdm import tqdm 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for param in model.features.parameters():
    param.requires_grad = False

model.to(device)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.NLLLoss()
EPOCHS = 3 

for e in range(EPOCHS):
   
  running_loss = 0

  for images, labels in tqdm(trainloader):

    optimizer.zero_grad()

    images, labels = images.to(device), labels.to(device)
    outputs = model.forward(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  else:


    model.eval()
    testing_loss = 0 
    acc = 0
    with torch.no_grad():
      for images, labels in tqdm(testloader):

        images, labels = images.to(device), labels.to(device)

        outputs = model.forward(images)
        loss = criterion(outputs, labels)

        top_k, top_label = outputs.topk(1, dim=1)
        equal = top_label == labels.view(*top_label.shape)
        acc += torch.mean(equal.type(torch.float))

        testing_loss += loss.item()
      
    print(f"\nEpoch {e+1}/{EPOCHS}..")
    print(f"Training loss : {running_loss/len(trainloader)}")
    print(f"Testing loss : {testing_loss/len(testloader)}")
    print(f"Accuracy : {100*acc/len(testloader)} %%")
    
    model.train()



  0%|          | 0/704 [00:00<?, ?it/s]


Epoch 1/3..
Training loss : 0.2963113956661387
Testing loss : 0.12948883645519427
Accuracy : 94.97626495361328 %%


  0%|          | 0/704 [00:00<?, ?it/s]


Epoch 2/3..
Training loss : 0.24963536822575738
Testing loss : 0.11405099246981024
Accuracy : 94.93671417236328 %%


100%|██████████| 79/79 [00:15<00:00,  5.23it/s]


Epoch 3/3..
Training loss : 0.24541761626658792
Testing loss : 0.11707901794321929
Accuracy : 95.41139221191406 %%
